In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import random

### Task 1

**Item 1**

Python generator of Bernoulli random variable. Able to generate infinitely. 


In [ ]:
def bern_gen(p):
    while True:
        if random.random() < p:
            yield 1
        else:
            yield 0

Python generator of binominal random variable. Able to generate infinitely.

In [ ]:
def bin_gen(p, n):
    bg = bern_gen(p)
    while True:
        sum = 0
        for i in range(n):
            sum += next(bg)
        yield sum

Visualization of binominal distribution with success probability *p* and number of attempts *n*.

In [ ]:
p = 0.3
n = 50
bing = bin_gen(p,n)
N = 1000

sns.set_theme()
values = {"Value" : [next(bing) for i in range(N)]}
sns.displot(data = pd.DataFrame(values), x = "Value", kde = True, kde_kws = {"bw_adjust" : 1}, discrete = True, stat = 'probability')

**Item 2**

Python generator of geometric random variable. Able to generate infinitely.

In [ ]:
def geom_gen(p):
    bg = bern_gen(p)
    while True:
        sum = 0
        while not next(bg):
            sum += 1
        yield sum

Visualization of geometric distribution.

In [ ]:
n = 10000
p = 0.3
gg = geom_gen(p)

sns.set_theme()
values = {"n" : [next(gg) for i in range(n)]}
sns.displot(data = pd.DataFrame(values), x = "n", kde = True, kde_kws = {"bw_adjust" : 2}, discrete = True, stat = 'probability')

Representation of geometric distribution memoryless property:  $\mathbb{P}(\mathbb{X} > m + n \, | \, \mathbb{X} > m) = \mathbb{P}(\mathbb{X} > n) $.  
(Previous cell execution required)

In [ ]:
m = 3

values[" "] = r'$\mathbb{P}(\mathbb{X} = n)$'

cutted_values = {"n" : [value - m for value in values["n"] if value >= m]}
cutted_values[" "] = r"$\mathbb{P}(\mathbb{X} = m + n | \mathbb{X} \geq m)$"

df_new_values = pd.concat([pd.DataFrame(values), pd.DataFrame(cutted_values)], ignore_index = True)

sns.set_theme()
sns.displot(data = df_new_values, x = "n", hue = " ", discrete = True, stat = 'probability', common_norm = False)